# Run the code in this notebook to finetune the model at a certain epoch

In [1]:
import torch
from torch.utils.data import DataLoader,Dataset
from diffusers import UNet2DConditionModel, DDPMScheduler, AutoencoderKL
from transformers import CLIPTokenizer, CLIPTextModel
from tqdm.auto import tqdm
import os
import json
from PIL import Image
from torchvision.transforms import transforms


In [5]:

# Parameters
batch_size = 8
EPOCHS = 2  # SET IT ACCORDINGLY
learning_rate = 1e-4
image_size = 128
latent_size = image_size // 16
data_path = "/teamspace/studios/this_studio/coco"
checkpoint_to_resume = "./ldm_checkpoints/epoch_4"  # Specify your checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizer and Transforms
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
max_length = 77



In [6]:
# Dataset class (same as your training script)
class CocoWithAnnotations(Dataset):
    def __init__(self, path, tokenizer, transform, train=True):
        super().__init__()
        self.path = path
        self.data = None
        self.transform = transform
        self.tokenizer = tokenizer
        self.train = train
        if self.data is None:
            self.open_json()

    def open_json(self):
        split = "train" if self.train else "val"
        with open(f'{self.path}/annotations/captions_{split}2014.json', 'r') as stream:
            self.data = json.load(stream)['annotations']

    def __getitem__(self, index):
        annot = self.data[index]
        img_id = str(annot['image_id']).zfill(12)
        split = "train" if self.train else "val"
        image_path = f'{self.path}/{split}2014/COCO_{split}2014_{img_id}.jpg'
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        text_emb = self.tokenizer(
            annot['caption'],
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        return image, text_emb.input_ids.squeeze(0)

    def __len__(self):
        return len(self.data)

# DataLoader
def collate_fn(batch):
    batch = [(img, cap) for img, cap in batch if img is not None and cap is not None]
    if not batch:
        return None
    images, captions = zip(*batch)
    return torch.stack(images), torch.stack(captions)

dataset = CocoWithAnnotations(data_path, tokenizer, transform, train=True)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

In [7]:
# Models
unet = UNet2DConditionModel.from_pretrained(os.path.join(checkpoint_to_resume, "unet")).to(device)
autoencoder = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(device)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
diffusion = DDPMScheduler(num_train_timesteps=1000, beta_schedule="linear")

# Freeze VAE and text encoder
for param in autoencoder.parameters():
    param.requires_grad = False
for param in text_encoder.parameters():
    param.requires_grad = False

# Optimizer
optimizer = torch.optim.AdamW(unet.parameters(), lr=learning_rate)
scaler = torch.cuda.amp.GradScaler()

# Load checkpoint
checkpoint = torch.load(os.path.join(checkpoint_to_resume, "training_state.pth"))
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
START_EPOCH = checkpoint['epoch'] + 1  # Start from the next epoch

# Training Loop
for epoch in range(START_EPOCH, START_EPOCH + EPOCHS):
    unet.train()
    epoch_loss = 0
    valid_batches = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{START_EPOCH + EPOCHS}")
    for step, batch in enumerate(progress_bar):
        if batch is None:
            print(f"[WARN] Batch {step}: NoneType batch skipped.")
            continue

        images, captions = batch
        images = images.to(device, non_blocking=True)
        captions = captions.to(device, non_blocking=True)

        # Sanity check on image inputs
        if not torch.isfinite(images).all():
            print(f"[ERROR] Non-finite values in images at step {step}, skipping batch.")
            continue

        with torch.cuda.amp.autocast():
            try:
                with torch.no_grad():
                    latents = autoencoder.encode(images).latent_dist.sample()
                    latents = latents * 0.18215

                if not torch.isfinite(latents).all():
                    print(f"[ERROR] NaN/Inf in latents at step {step}, skipping batch.")
                    continue

                noise = torch.randn_like(latents)
                timesteps = torch.randint(
                    0, diffusion.config.num_train_timesteps, 
                    (latents.shape[0],), device=device
                )

                noisy_latents = diffusion.add_noise(latents, noise, timesteps)

                if not torch.isfinite(noisy_latents).all():
                    print(f"[ERROR] NaN/Inf in noisy_latents at step {step}, skipping batch.")
                    continue

                with torch.no_grad():
                    encoder_hidden_states = text_encoder(captions)[0]

                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            except Exception as e:
                print(f"[EXCEPTION] Error at step {step}: {e}")
                continue

        # Force loss to run in full precision (helps avoid AMP overflow)
        with torch.cuda.amp.autocast(enabled=False):
            loss = torch.nn.functional.mse_loss(
                noise_pred.float(), noise.float(), reduction="mean"
            )

        # Detect NaN loss early
        if not torch.isfinite(loss):
            print(f"[WARN] Non-finite loss detected (step {step}), skipping optimizer step.")
            optimizer.zero_grad(set_to_none=True)
            scaler = torch.cuda.amp.GradScaler()
            continue

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(unet.parameters(), 0.5)

        try:
            scaler.step(optimizer)
            scaler.update()
        except Exception as e:
            print(f"[EXCEPTION] Optimizer/Scaler step failed at step {step}: {e}")
            optimizer.zero_grad(set_to_none=True)
            continue

        optimizer.zero_grad(set_to_none=True)

        epoch_loss += loss.item()
        valid_batches += 1
        progress_bar.set_postfix({"loss": f"{loss.item():.6f}"})

        # Optional: print detailed debug info occasionally
        if step % 500 == 0:
            print(f"[DEBUG] Step {step}: loss={loss.item():.6f}, scale={scaler.get_scale()}")

    avg_loss = epoch_loss / max(valid_batches, 1)
    print(f"Epoch {epoch + 1} completed with average loss: {avg_loss:.6f}")

    # Save checkpoint every epoch
    checkpoint_dir = f"./ldm_checkpoints/epoch_{epoch + 1}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    unet.save_pretrained(os.path.join(checkpoint_dir, "unet"))
    torch.save({
        'epoch': epoch,
        'model_state_dict': unet.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
    }, os.path.join(checkpoint_dir, "training_state.pth"))

print("Training resumed and completed!")



Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/tmp/ipykernel_2841/2089717187.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch 5/6:   0%|          | 0/51765 [00:00<?, ?it/s]

/tmp/ipykernel_2841/2089717187.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipykernel_2841/2089717187.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


[DEBUG] Step 0: loss=0.040160, scale=65536.0
[DEBUG] Step 500: loss=0.103748, scale=65536.0
[DEBUG] Step 1000: loss=0.113798, scale=65536.0
[DEBUG] Step 1500: loss=0.379164, scale=65536.0
[DEBUG] Step 2000: loss=0.181589, scale=131072.0
[DEBUG] Step 2500: loss=0.144979, scale=131072.0
[WARN] Non-finite loss detected (step 2662), skipping optimizer step.
[WARN] Non-finite loss detected (step 2663), skipping optimizer step.


/tmp/ipykernel_2841/2089717187.py:84: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


[WARN] Non-finite loss detected (step 2664), skipping optimizer step.
[WARN] Non-finite loss detected (step 2665), skipping optimizer step.
[WARN] Non-finite loss detected (step 2666), skipping optimizer step.
[WARN] Non-finite loss detected (step 2667), skipping optimizer step.
[WARN] Non-finite loss detected (step 2668), skipping optimizer step.
[WARN] Non-finite loss detected (step 2669), skipping optimizer step.
[WARN] Non-finite loss detected (step 2670), skipping optimizer step.
[WARN] Non-finite loss detected (step 2671), skipping optimizer step.
[WARN] Non-finite loss detected (step 2672), skipping optimizer step.
[WARN] Non-finite loss detected (step 2673), skipping optimizer step.
[WARN] Non-finite loss detected (step 2674), skipping optimizer step.
[WARN] Non-finite loss detected (step 2675), skipping optimizer step.
[WARN] Non-finite loss detected (step 2676), skipping optimizer step.
[WARN] Non-finite loss detected (step 2677), skipping optimizer step.
[WARN] Non-finite lo

Epoch 6/6:   0%|          | 0/51765 [00:00<?, ?it/s]

[DEBUG] Step 0: loss=0.218787, scale=524288.0
[DEBUG] Step 500: loss=0.181844, scale=524288.0
[DEBUG] Step 1000: loss=0.117297, scale=524288.0
[DEBUG] Step 1500: loss=0.097832, scale=1048576.0
[DEBUG] Step 2000: loss=0.128336, scale=1048576.0
[DEBUG] Step 2500: loss=0.356366, scale=524288.0
[DEBUG] Step 3000: loss=0.042921, scale=524288.0
[DEBUG] Step 3500: loss=0.139375, scale=524288.0
[DEBUG] Step 4000: loss=0.105939, scale=524288.0
[DEBUG] Step 4500: loss=0.265950, scale=1048576.0
[DEBUG] Step 5000: loss=0.234737, scale=524288.0
[DEBUG] Step 5500: loss=0.027198, scale=524288.0
[DEBUG] Step 6000: loss=0.065229, scale=524288.0
[DEBUG] Step 6500: loss=0.026829, scale=524288.0
[DEBUG] Step 7000: loss=0.068597, scale=1048576.0
[DEBUG] Step 7500: loss=0.313256, scale=1048576.0
[DEBUG] Step 8000: loss=0.044320, scale=524288.0
[DEBUG] Step 8500: loss=0.062602, scale=524288.0
[DEBUG] Step 9000: loss=0.196734, scale=524288.0
[DEBUG] Step 9500: loss=0.260330, scale=262144.0
[DEBUG] Step 10000: